In [1]:
import os
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
import keras
from keras import backend as k
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense,Dropout,Activation
from keras.optimizers import Adam
#from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
import argparse
from keras.layers.normalization import BatchNormalization
#from pyimagesearch.stridednet import StridedNet
from keras import backend as K
from sklearn.preprocessing import LabelBinarizer


Using TensorFlow backend.


In [2]:
def resize(img):
    img = cv2.resize(img,(20,20))
    return img

In [3]:
X_train = []
y_train = []

# to get the name of the folder
for name_folder in os.listdir("C:\\Users\HP\Desktop\optical-character-recognition-OCR-master\Fnt") :

    name = 'C:\\Users\HP\Desktop\optical-character-recognition-OCR-master\Fnt/' + name_folder
    for f in listdir(name):
        # name of the folder is the name of the output
        y_train.append(np.asarray(name_folder))
        
        # constructing full path to the image
        name = 'C:\\Users\HP\Desktop\optical-character-recognition-OCR-master\Fnt/' + name_folder + '/' + f
        
        # reading the image
        image = cv2.imread(name,0)/255
        
        # appending to form the image list
        image = np.asarray(image)
        image = resize(image)
        X_train.append([image])

In [4]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_train.shape

(36576, 1, 20, 20)

In [5]:
X_train = np.reshape(X_train, [-1,20,20,1])

In [6]:
X_train.shape

(36576, 20, 20, 1)

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_train)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_train = onehot_encoder.fit_transform(integer_encoded)

In [8]:
class StridedNet:
    def build(width, height, depth,reg, init="he_normal"):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        
		# if we are using "channels first", update the input shape
		# and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
            
# our first CONV layer will learn a total of 16 filters, each
		# Of which are 7x7 -- we'll then apply 2x2 strides to reduce
		# the spatial dimensions of the volume
        model.add(Conv2D(16, (7, 7), strides=(2, 2), padding="valid",
            kernel_initializer=init, kernel_regularizer=reg,
            input_shape=inputShape))
        
		# here we stack two CONV layers on top of each other where
		# each layerswill learn a total of 32 (3x3) filters
        model.add(Conv2D(32, (3, 3), padding="same",
            kernel_initializer=init, kernel_regularizer=reg))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(32, (3, 3), strides=(2, 2), padding="same",
            kernel_initializer=init, kernel_regularizer=reg))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.25))
        
        # stack two more CONV layers, keeping the size of each filter
		# as 3x3 but increasing to 64 total learned filters
        model.add(Conv2D(64, (3, 3), padding="same",
            kernel_initializer=init, kernel_regularizer=reg))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), strides=(2, 2), padding="same",
            kernel_initializer=init, kernel_regularizer=reg))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.25))
        
		# increase the number of filters again, this time to 128
        model.add(Conv2D(128, (3, 3), padding="same",
            kernel_initializer=init, kernel_regularizer=reg))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), strides=(2, 2), padding="same",
            kernel_initializer=init, kernel_regularizer=reg))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.25))
        
        # fully-connected layer
        model.add(Flatten())
        model.add(Dense(512, kernel_initializer=init))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
		# softmax classifier
        model.add(Dense(36, activation="softmax"))
        return model


In [9]:

    

#opt = Adam(lr=1e-4, decay=1e-4 / args[50])
model = StridedNet.build(width=20, height=20, depth=1,reg=l2(0.0005))
#model=StridedNet()
model.compile(loss="categorical_crossentropy", optimizer="Adam",metrics=["accuracy"])



In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 7, 7, 16)          800       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
activation_1 (Activation)    (None, 7, 7, 32)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 32)          128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 32)          9248      
_________________________________________________________________
activation_2 (Activation)    (None, 4, 4, 32)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 4, 4, 32)         

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train ,test_size = .2)

In [12]:
X_train.shape

(29260, 20, 20, 1)

In [14]:
X_test.shape

(7316, 20, 20, 1)

In [13]:
y_train.shape

(29260, 36)

In [14]:
print(len(X_train))

29260


In [15]:
# fitting the model
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	  width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	  horizontal_flip=True, fill_mode="nearest")

model.fit_generator(aug.flow(X_train, y_train, batch_size=300),
                             steps_per_epoch=len(X_train) // 300,
                             epochs=50,
                             validation_data=(X_test,y_test), shuffle = 1)

Epoch 1/50
97/97 [==============================] - 38s 396ms/step - loss: 4.7930 - accuracy: 0.0440 - val_loss: 3.8703 - val_accuracy: 0.0428
Epoch 2/50
97/97 [==============================] - 38s 395ms/step - loss: 4.0529 - accuracy: 0.0774 - val_loss: 3.3223 - val_accuracy: 0.1588
Epoch 3/50
97/97 [==============================] - 37s 380ms/step - loss: 3.5389 - accuracy: 0.1368 - val_loss: 3.1382 - val_accuracy: 0.2276
Epoch 4/50
97/97 [==============================] - 38s 396ms/step - loss: 3.1535 - accuracy: 0.1972 - val_loss: 2.5913 - val_accuracy: 0.3252
Epoch 5/50
97/97 [==============================] - 29s 295ms/step - loss: 2.8634 - accuracy: 0.2552 - val_loss: 2.2502 - val_accuracy: 0.4258
Epoch 6/50
97/97 [==============================] - 28s 292ms/step - loss: 2.6218 - accuracy: 0.3186 - val_loss: 2.0474 - val_accuracy: 0.4993
Epoch 7/50
97/97 [==============================] - 28s 285ms/step - loss: 2.4195 - accuracy: 0.3681 - val_loss: 1.9072 - val_accuracy: 0.5252

In [16]:
#saving the model 
model.save('C:\\Users\HP\Desktop\optical-character-recognition-OCR-master\models\model45.h5')

In [17]:
np.save('C:\\Users\HP\Desktop\optical-character-recognition-OCR-master\models/xtrain',X_train,allow_pickle=True)